In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

In [3]:
def check_availability(park_name, trail_name, target_date):
    try:
        # Initialize the WebDriver
        driver = webdriver.Chrome()
        driver.get("https://reserve.bcparks.ca/dayuse/")

        park_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//button[contains(@aria-label, '{park_name}')]"))
        )

        # Click the park button
        park_button.click()

        # Click on the date field
        date_field = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/app-root/div/div[2]/app-registration/div/div/div/div/div[2]/app-facility-select/form/div/app-date-picker/fieldset"))
        )
        date_field.click()

        # Click on a specific date in the date picker
        date_selector = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/app-root/div/div[2]/app-registration/div/div/div/div/div[2]/app-facility-select/form/div/app-date-picker/fieldset/ngb-datepicker/div[2]/div/ngb-datepicker-month/div[4]/div[7]"))
        )
        date_selector.send_keys(target_date)
        date_selector.click()

        # Select a hike from the dropdown menu
        hike_select = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//select//option[contains(., '{trail_name}')]"))
        )
        hike_select.click()

        booking_options = driver.find_elements(By.CLASS_NAME, "card-header")
        available = False
        for b in booking_options:
            if 'card-header-disabled' not in b.get_attribute('class'):
                available = True
        print(f"{park_name} - {trail_name} - {available}")
        return available
    finally:
        driver.quit()

check_availability(park_name="Joffre", trail_name="Joffre", target_date="2024-07-23")

ElementClickInterceptedException: Message: element click intercepted: Element <div role="gridcell" class="ngb-dp-day disabled" tabindex="-1" aria-label="Saturday, July 20, 2024">...</div> is not clickable at point (869, 496). Other element would receive the click: <div role="row" class="ngb-dp-week">...</div>
  (Session info: chrome=126.0.6478.183)
Stacktrace:
0   chromedriver                        0x000000010452aa0c chromedriver + 4385292
1   chromedriver                        0x0000000104523318 chromedriver + 4354840
2   chromedriver                        0x0000000104140b0c chromedriver + 281356
3   chromedriver                        0x0000000104188ce0 chromedriver + 576736
4   chromedriver                        0x0000000104187284 chromedriver + 569988
5   chromedriver                        0x0000000104185414 chromedriver + 562196
6   chromedriver                        0x00000001041845b0 chromedriver + 558512
7   chromedriver                        0x0000000104179cfc chromedri

In [ ]:
import smtplib
from email.mime.text import MIMEText

def send_email_notifications(to_email, subject, message):
    from_email = os.getend('USERNAME')
    password = os.getenv('USERNAME')

    msg = MIMEText(message)
    msg['Subject'] = subject
    msg['From'] = from_email
    msg['To'] = to_email

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
        server.login(from_email, password)
        server.sendmail(from_email, to_email, msg.as_string())

def notify():
    subject = 'GO GO GO'
    message = 'The passes are now available! Check the website.'
    send_email_notifications('sk22.social@gmail.com', subject, message)

In [ ]:
import schedule
import time

def job():
    if check_availability(park_name="Joffre", trail_name="Joffre", target_date="2024-07-23"):
        notify()

schedule.every(5).minutes.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)